In [9]:
# Required Modules
import pandas as pd
import numpy as np


In [29]:
data = pd.read_csv('Resources/Final_Data.csv')
data.head()

,Zip_Code,Date,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,Rent_Affordability,...,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
0,32003,2019-01-01,2019,1,244950.00,4.46,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,195960.00,360.00,1.00,988.71,0.13
1,32003,2019-02-01,2019,2,270000.00,4.37,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,216000.00,360.00,1.00,1077.82,0.14
2,32003,2019-03-01,2019,3,275000.00,4.26,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,220000.00,360.00,1.00,1084.20,0.14
3,32003,2019-04-01,2019,4,264500.00,4.14,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,211600.00,360.00,1.00,1027.67,0.13
4,32003,2019-05-01,2019,5,281000.00,4.07,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,224800.00,360.00,1.00,1082.58,0.14


In [30]:
data.dtypes

Zip_Code                int64
Date                   object
Year                    int64
Month                   int64
Sale_Price            float64
Interest_Rate         float64
Property_Tax            int64
Rent_Price              int64
Household_Income      float64
Rent_Affordability    float64
Total_Sales             int64
FHA_Count               int64
Mobility_Rate         float64
School_Rating           int64
School_Grade           object
Owner_Occupied          int64
Renter_Occupied         int64
Total_Vacant            int64
Total_Dwellings         int64
FTE_Employed            int64
Unemployed              int64
Expense_Index           int64
Average_Commute         int64
Crime_Index             int64
Loan_Amount           float64
Loan_Term             float64
Loan_R                float64
Loan_Payment          float64
Home_Affordability    float64
dtype: object

In [48]:
ml_data = data[['Zip_Code', 'Year', 'Month', 'Total_Sales', 'FHA_Count', 
'Home_Affordability', 'Rent_Affordability', 'Sale_Price' ]]
ml_data

,Zip_Code,Year,Month,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,32,7,0.13,0.14,244950.00
1,32003,2019,2,41,2,0.14,0.14,270000.00
2,32003,2019,3,21,7,0.14,0.14,275000.00
3,32003,2019,4,42,3,0.13,0.14,264500.00
4,32003,2019,5,66,8,0.14,0.14,281000.00
...,...,...,...,...,...,...,...,...
21759,34997,2020,10,139,17,0.19,0.25,285000.00
21760,34997,2020,11,132,11,0.17,0.25,259750.00
21761,34997,2020,11,132,11,0.17,0.25,259750.00
21762,34997,2020,12,142,16,0.20,0.25,297000.00


In [33]:
count = np.isinf(ml_data).values.sum()
print("It contains " + str(count) + " infinite values")

It contains 0 infinite values


In [34]:
# # Replacing infinite with nan
# ml_data.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# # Dropping all the rows with nan values
# ml_data.dropna(inplace=True)
  
# # Printing df
# ml_data

In [49]:
ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data

,Zip_Code,Year,Month,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,32,7,0.13,0.14,244950.00
1,32003,2019,2,41,2,0.14,0.14,270000.00
2,32003,2019,3,21,7,0.14,0.14,275000.00
3,32003,2019,4,42,3,0.13,0.14,264500.00
4,32003,2019,5,66,8,0.14,0.14,281000.00
...,...,...,...,...,...,...,...,...
21759,34997,2020,10,139,17,0.19,0.25,285000.00
21760,34997,2020,11,132,11,0.17,0.25,259750.00
21761,34997,2020,11,132,11,0.17,0.25,259750.00
21762,34997,2020,12,142,16,0.20,0.25,297000.00


In [50]:
ml_data['Last_Month_Price'] = ml_data.groupby(['Zip_Code'])['Sale_Price'].shift()



price_difference = []

# for (Sale_Price, Last_Month_Price) in test.iteritems():
#     # sale = row.Sale_Price
#     # last = row.Last_Month_Price
#     price_difference.append(row)

for column in ml_data[['Sale_Price', 'Last_Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = ml_data['Sale_Price'].values
    last_sales = ml_data['Last_Month_Price'].values
    price_difference.append(current_sales-last_sales)

ml_data['Last_Month_Diff'] = price_difference[0]

price_change = []

for row in ml_data['Last_Month_Diff']:
    if row > 0:
        price_change.append(1)
    else:
        price_change.append(0)
ml_data['Price_Change_1mo'] = price_change

ml_data

,Zip_Code,Year,Month,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_Month_Diff,Price_Change_1mo
0,32003,2019,1,32,7,0.13,0.14,244950.00,nan,nan,0
1,32003,2019,2,41,2,0.14,0.14,270000.00,244950.00,25050.00,1
2,32003,2019,3,21,7,0.14,0.14,275000.00,270000.00,5000.00,1
3,32003,2019,4,42,3,0.13,0.14,264500.00,275000.00,-10500.00,0
4,32003,2019,5,66,8,0.14,0.14,281000.00,264500.00,16500.00,1
...,...,...,...,...,...,...,...,...,...,...,...
21759,34997,2020,10,139,17,0.19,0.25,285000.00,285000.00,0.00,0
21760,34997,2020,11,132,11,0.17,0.25,259750.00,285000.00,-25250.00,0
21761,34997,2020,11,132,11,0.17,0.25,259750.00,259750.00,0.00,0
21762,34997,2020,12,142,16,0.20,0.25,297000.00,259750.00,37250.00,1


In [38]:
import warnings
warnings.filterwarnings("ignore")

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [18]:
y_value = test['Sale_Price'].values

x_values = test.drop(columns='Sale_Price')

feature_names = x_values.columns

x_values
y_value

array([244950., 270000., 275000., ..., 259750., 297000., 297000.])

In [19]:

X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [20]:
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Test set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (16323, 10) Target:  (16323,)
Test set - Features:  (5441, 10) Target:  (5441,)


In [21]:
import lazypredict
from lazypredict.Supervised import LazyRegressor

In [22]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
reg_models, reg_predictions = reg.fit(X_train, X_test, y_train, y_test)

print(reg_models)
print(reg_predictions)

100%|██████████| 42/42 [01:34<00:00,  2.24s/it]                               Adjusted R-Squared  R-Squared       RMSE  \
Model                                                                     
LassoLarsCV                                  0.97       0.97   25774.54   
LassoLarsIC                                  0.97       0.97   25774.54   
TransformedTargetRegressor                   0.97       0.97   25774.54   
LarsCV                                       0.97       0.97   25774.54   
Lars                                         0.97       0.97   25774.54   
LinearRegression                             0.97       0.97   25774.54   
Lasso                                        0.97       0.97   25774.63   
BayesianRidge                                0.97       0.97   25775.00   
Ridge                                        0.97       0.97   25775.72   
SGDRegressor                                 0.97       0.97   25780.64   
LassoCV                                      0.97    

In [51]:
ml_data = ml_data.drop(columns=['Last_Month_Diff', 'Last_Month_Price'])
ml_data

,Zip_Code,Year,Month,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Price_Change_1mo
0,32003,2019,1,32,7,0.13,0.14,244950.00,0
1,32003,2019,2,41,2,0.14,0.14,270000.00,1
2,32003,2019,3,21,7,0.14,0.14,275000.00,1
3,32003,2019,4,42,3,0.13,0.14,264500.00,0
4,32003,2019,5,66,8,0.14,0.14,281000.00,1
...,...,...,...,...,...,...,...,...,...
21759,34997,2020,10,139,17,0.19,0.25,285000.00,0
21760,34997,2020,11,132,11,0.17,0.25,259750.00,0
21761,34997,2020,11,132,11,0.17,0.25,259750.00,0
21762,34997,2020,12,142,16,0.20,0.25,297000.00,1


In [52]:
y_value = ml_data['Price_Change_1mo'].values

x_values = ml_data.drop(columns='Price_Change_1mo')

feature_names = x_values.columns

x_values
y_value

array([0, 1, 1, ..., 0, 1, 0])

In [53]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [54]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=False)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 21%|██        | 6/29 [00:04<00:17,  1.32it/s]CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)
 90%|████████▉ | 26/29 [01:25<00:11,  3.89s/it]StackingClassifier model failed to execute
__init__() missing 1 required positional argument: 'estimators'
100%|██████████| 29/29 [01:26<00:00,  2.97s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.66,0.67,0.67,0.66,0.44
XGBClassifier,0.65,0.65,0.65,0.65,0.58
SVC,0.65,0.65,0.65,0.65,7.29
AdaBoostClassifier,0.64,0.64,0.64,0.64,0.50
RandomForestClassifier,0.64,0.64,0.64,0.64,2.04
BaggingClassifier,0.62,0.62,0.62,0.62,0.60
LogisticRegression,0.61,0.61,0.61,0.61,0.04
CalibratedClassifierCV,0.61,0.61,0.61,0.61,3.67
ExtraTreesClassifier,0.61,0.61,0.61,0.61,1.26
